# Projekt
**Filip Jakovina, 0036535147**
## A Machine Learning Strategy For Predicting March Madness Winnerss

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Priprema i vizualizacija podataka

## Učitavanje podataka

In [4]:
rsr = pd.read_csv("data/regular_season_results.csv",index_col = 0)
ss = pd.read_csv("data/sample_submission.csv",index_col = 0)
s = pd.read_csv("data/seasons.csv",index_col = 0)
t = pd.read_csv("data/teams.csv",index_col = 0)
tr = pd.read_csv("data/tourney_results.csv",index_col = 0)
tse = pd.read_csv("data/tourney_seeds.csv",index_col = 0)
tsl = pd.read_csv("data/tourney_slots.csv",index_col = 0)

## Prikaz i analiza podataka

### Sezone
Popis sezona

In [10]:
with pd.option_context("display.max_rows", None, 'display.max_columns', None):
    display(s)

,years,dayzero,regionW,regionX,regionY,regionZ
season,,,,,,
A,1995-1996,10/30/1995,East,Midwest,Southeast,West
B,1996-1997,10/28/1996,East,Southeast,Midwest,West
C,1997-1998,10/27/1997,East,West,Midwest,South
D,1998-1999,10/26/1998,East,Midwest,South,West
E,1999-2000,11/01/1999,Midwest,West,East,South
F,2000-2001,10/30/2000,East,West,Midwest,South
G,2001-2002,10/29/2001,East,Midwest,South,West
H,2002-2003,11/04/2002,East,South,Midwest,West
I,2003-2004,11/03/2003,Atlanta,Phoenix,EastRutherford,StLouis


Pojašnjenja značajki:
- Season &ndash; jedno-znakovni identifikator sezone
- Years &ndash; godine kroz koje se proteže sezona
- Dayzero &ndash; datum početka sezone
- RegionW/X/Y/Z &ndash; 4 regije u završnom turniru sezone

In [11]:
s.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, A to S
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   years    19 non-null     object
 1   dayzero  19 non-null     object
 2   regionW  19 non-null     object
 3   regionX  19 non-null     object
 4   regionY  19 non-null     object
 5   regionZ  19 non-null     object
dtypes: object(6)
memory usage: 1.0+ KB


In [12]:
s.describe(include='all')

,years,dayzero,regionW,regionX,regionY,regionZ
count,19,19,19,19,19,19
unique,19,19,4,7,7,6
top,1995-1996,10/30/1995,East,South,Midwest,West
freq,1,1,15,6,9,12


### Timovi
Popis identifikatora i imena timova

In [13]:
with pd.option_context("display.max_rows", 10, 'display.max_columns', None):
    display(t)

,name
id,
501,Abilene Chr
502,Air Force
503,Akron
504,Alabama
505,Alabama A&M
...,...
852,Wyoming
853,Xavier
854,Yale


In [14]:
t.describe(include='all')

,name
count,356
unique,356
top,Abilene Chr
freq,1


### Rezultati regularne sezone
Rezultati svake utakmice u regularnom dijelu sezone 

In [6]:
with pd.option_context("display.max_rows", 10, 'display.max_columns', None):
    display(rsr)

,daynum,wteam,wscore,lteam,lscore,wloc,numot
season,,,,,,,
A,16,511,91,647,57,H,NaN
A,16,515,75,812,67,H,NaN
A,16,606,87,658,67,H,NaN
A,16,670,73,573,65,H,NaN
A,16,721,99,632,68,H,NaN
...,...,...,...,...,...,...,...
S,132,592,61,640,60,N,0.0
S,132,671,69,670,55,N,0.0
S,132,779,65,825,61,N,0.0


Pojašnjenja značajki:
 - Season &ndash; jedno-znakovni identifikator sezone (odgovara stupcu "season" u datoteci "seasons.csv")
 - Daynum &ndash; broj dana nakon "dayzero" datuma u "seasons.csv"
 - Wteam &ndash; Id pobjedničke momčadi iz "teams.csv" 
 - Wscore &ndash; Broj poena koje je pobjednik postigao
 - Lteam &ndash; Id gubitničke momčadi
 - Lscore &ndash; Broj poena koje je gubitnik postigao
 - Wloc &ndash; Lokacija pobjedničke momčadi (H - domaćin, A - gost, N - neutralan teren)
 - Numot &ndash; Broj odigranih produžetaka na utakmici

In [9]:
rsr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 91224 entries, A to S
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   daynum  91224 non-null  int64  
 1   wteam   91224 non-null  int64  
 2   wscore  91224 non-null  int64  
 3   lteam   91224 non-null  int64  
 4   lscore  91224 non-null  int64  
 5   wloc    91224 non-null  object 
 6   numot   51333 non-null  float64
dtypes: float64(1), int64(5), object(1)
memory usage: 5.6+ MB


In [7]:
rsr.describe(include='all')

,daynum,wteam,wscore,lteam,lscore,wloc,numot
count,91224.000000,91224.000000,91224.000000,91224.000000,91224.000000,91224,51333.000000
unique,NaN,NaN,NaN,NaN,NaN,3,NaN
top,NaN,NaN,NaN,NaN,NaN,H,NaN
freq,NaN,NaN,NaN,NaN,NaN,54891,NaN
mean,73.410133,680.981058,75.370440,677.047400,63.208860,NaN,0.072000
std,34.210997,103.099502,11.408528,102.950054,10.940295,NaN,0.313278
min,0.000000,501.000000,34.000000,501.000000,20.000000,NaN,0.000000
25%,44.000000,592.000000,68.000000,587.000000,56.000000,NaN,0.000000
50%,75.000000,678.000000,75.000000,675.000000,63.000000,NaN,0.000000
75%,103.000000,772.000000,83.000000,768.000000,70.000000,NaN,0.000000


### Rezultati završnog turnira sezone
Rezulati svake utakmice u završnom turniru sezone

In [15]:
with pd.option_context("display.max_rows", 10, 'display.max_columns', None):
    display(tr)

,daynum,wteam,wscore,lteam,lscore,numot
season,,,,,,
A,136,515,86,729,80,NaN
A,136,559,68,555,59,NaN
A,136,576,75,666,63,NaN
A,136,581,75,577,60,NaN
A,136,604,81,551,74,NaN
...,...,...,...,...,...,...
R,146,651,85,577,63,0.0
R,146,670,79,592,59,0.0
R,152,651,72,846,68,0.0


Značajke su iste kao i u prethodnoj tablici za regularni dio sezone <br>
Jedino nema značajke "Wloc", jer se podrazumijeva da se svaka utakmica u završnom
turniru sezone igra na neutralnom terenu

In [17]:
tr.describe(include='all')

,daynum,wteam,wscore,lteam,lscore,numot
count,1156.000000,1156.000000,1156.000000,1156.000000,1156.000000,585.000000
mean,139.143599,682.515571,75.549308,686.699827,63.667820,0.075214
std,4.213580,98.684397,10.941410,103.201450,10.567058,0.311565
min,134.000000,504.000000,43.000000,502.000000,29.000000,0.000000
25%,136.000000,606.000000,68.000000,603.750000,57.000000,0.000000
50%,137.000000,671.000000,75.000000,681.500000,63.000000,0.000000
75%,139.000000,783.000000,82.000000,783.000000,70.000000,0.000000
max,154.000000,853.000000,121.000000,853.000000,105.000000,2.000000


### Sjemena završnog turnira
Identifikacija sjemena za 64 tima u svakom zavšnom turniru

In [18]:
with pd.option_context("display.max_rows", 10, 'display.max_columns', None):
    display(tse)

,seed,team
season,,
A,W01,663
A,W02,603
A,W03,796
A,W04,660
A,W05,729
...,...,...
S,Z12,689
S,Z13,702
S,Z14,811


Pojašnjenja značajki:
 - Season &ndash; jedno-znakovni identifikator sezone
 - Seed &ndash; troznakovni identifikator sjemena
 - Team &ndash; Id momčadi

In [19]:
tse.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1242 entries, A to S
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   seed    1242 non-null   object
 1   team    1242 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 29.1+ KB


In [20]:
tse.describe(include='all')

,seed,team
count,1242,1242.000000
unique,86,NaN
top,W01,NaN
freq,19,NaN
mean,NaN,685.844605
std,NaN,103.133122
min,NaN,502.000000
25%,NaN,603.000000
50%,NaN,680.000000
75%,NaN,783.000000


### Turnirska uparivanja
Popis uparivanja utakmica u završnom turniru

In [21]:
with pd.option_context("display.max_rows", 10, 'display.max_columns', None):
    display(tsl)

,slot,strongseed,weakseed
season,,,
A,R1W1,W01,W16
A,R1W2,W02,W15
A,R1W3,W03,W14
A,R1W4,W04,W13
A,R1W5,W05,W12
...,...,...,...
S,R4Y1,R3Y1,R3Y2
S,R4Z1,R3Z1,R3Z2
S,R5WX,R4W1,R4X1


Pojašnjenja značajki:
 - Season &ndash; jedno-znakovni identifikator sezone
 - Slot &ndash; identifikator uparivanja timova u završnom turniru po rundama
 - Strongseed &ndash; sjeme momčadi ili pobjednika prošle runde za koje se smatra da će pobijediti u ovoj rundi
 - Weakseed &ndash; sjeme momčadi ili pobjednika prošle runde za koje se smatra da će izgubiti u ovoj rundi

In [22]:
tsl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1223 entries, A to S
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   slot        1223 non-null   object
 1   strongseed  1223 non-null   object
 2   weakseed    1223 non-null   object
dtypes: object(3)
memory usage: 38.2+ KB


In [23]:
tsl.describe(include='all')

,slot,strongseed,weakseed
count,1223,1223,1223
unique,74,74,74
top,R1W1,W01,W16
freq,19,19,19
